# Imports and Custom Modules

This section of the notebook imports essential libraries and custom modules required for data analysis and visualization:

- **Standard Libraries**:
    - `sys`: Provides access to system-specific parameters and functions.
    - `pandas`: Used for data manipulation and analysis.
    - `random`: Generates random numbers for simulation purposes.
    - `datetime`: Handles date and time operations.

- **Custom Modules**:
    - `generate_signals`: A custom module for generating trading signals based on stock data.
    - `generate_candlestick_df`: A custom module for formatting data for candlestick chart generation.

In [15]:
import sys
import pandas as pd
import random
from datetime import datetime, timedelta
from generate_signals import generate_signals  # Importing signal generation function

# Setup and Customization Variables

This section provides a brief description of the key variables used for configuring and running the program:

1. **Timeframe and Date Variables**:
    - `start_date` and `end_date`: Define the simulation's date range.
    - `random_start_date` and `random_dates`: Specify random start dates for the simulation.
    
2. **Market Session Variables**:
    - `market_open`, `market_close`, and `pre_market_start`: Define the pre-market, open, and close times.

3. **Simulation Parameters**:
    - `number_of_days`: Number of days to plot and retrieve data for.
    - `intervalAmt`: Time interval for candlestick data aggregation.

In [16]:
# Generate a random start date within the range
start_date = datetime(2024, 12, 1)
end_date = datetime(2025, 1, 31)
number_of_days = 62
intervalAmt = 60
# Define market session times
market_open = '09:30:00'
market_close = '16:00:00'
pre_market_start = '04:00:00'

# Calculate the maximum possible start date to ensure the range fits `number_of_days`
max_start_date = end_date - timedelta(days=number_of_days - 1)
random_start_date = start_date + timedelta(days=random.randint(0, (max_start_date - start_date).days))

# Generate continuous dates
random_dates = [random_start_date + timedelta(days=i) for i in range(number_of_days)]

# Data Reformatting and Indicator Generation

The cell below is designed to process and analyze stock market data for visualization and trading signal generation. It performs the following key functions:

1. **Data Filtering and Transformation**:
  - Filters a large dataset (`df`) to extract relevant rows based on specific dates (`formatted_random_dates`).
  - Converts and reformats columns (e.g., `Datetime` to `Timestamp`) for consistency and ease of analysis.

2. **Technical Indicator Calculation**:
  - Computes key indicators like:
    - **8EMA**: Exponential Moving Average over an 8-period timeframe.
    - **VWAP**: Volume Weighted Average Price.
  - Adds these indicators as new columns to the filtered dataset (`retrieved_data`).

3. **Session and Day Segmentation**:
  - Segments data into distinct trading sessions (e.g., pre-market, regular market) using timestamps.
  - Maps each row to a specific trading day using a `day_mapping` dictionary.

4. **Key Levels Calculation**:
  - Calculates critical price levels for each trading day:
    - **ORB High/Low**: Opening Range Breakout levels (first 15 minutes of market open).
    - **PM High/Low**: Pre-market high and low prices.
    - **Yesterday's High/Low**: Previous day's high and low prices.
  - Stores these levels in lists (`orb_highs`, `orb_lows`, `pm_highs`, `pm_lows`, `yest_highs`, `yest_lows`) and maps them back to the dataset.

5. **Signal Generation**:
  - Applies a custom signal generation function (`generate_signals`) to identify trading opportunities (e.g., "BUY CALL", "BUY PUT") and calculate stop-loss levels.

### Key Variables:
- **`retrieved_data`**: The main DataFrame containing filtered and processed stock data with added indicators, session information, and trading signals.
- **`orb_highs`, `orb_lows`, `pm_highs`, `pm_lows`, `yest_highs`, `yest_lows`**: Lists storing calculated price levels for each trading day.
- **`day_mapping`**: A dictionary mapping unique dates to sequential day indices.
- **`formatted_random_dates`**: A list of dates used to filter the dataset.
- **`generate_signals`**: A custom function applied to generate trading signals and stop-loss levels.

This cell prepares the data for further analysis, visualization, and decision-making in the context of stock trading. It ensures the data is enriched with technical indicators and key levels, making it suitable for candlestick chart generation and trading strategy evaluation.

In [17]:
# Load the CSV file into a DataFrame and capitalize the first letter of column titles
df = pd.read_csv('df_2023_2025.csv')
df.columns = [col.capitalize() for col in df.columns]

# Convert the 'datetime' column to string
df['Datetime'] = df['Datetime'].astype(str)

# Reformat random_dates to match the format in the 'datetime' column
formatted_random_dates = [date.strftime('%Y-%m-%d') for date in random_dates]

# Filter the dataframe to find rows where the 'datetime' column starts with any of the formatted_random_dates
retrieved_data = df[df['Datetime'].str.startswith(tuple(formatted_random_dates))]

# Read the volume data from the CSV file
volume_data = pd.read_csv('df_2023_2025_volumes.csv', usecols=['volume'])

# Add the volume column to retrieved_data and rename it to "Volume"
retrieved_data['Volume'] = volume_data['volume']

# Calculate 8EMA (Exponential Moving Average over an 8-period timeframe)
retrieved_data['8EMA'] = retrieved_data['Close'].ewm(span=8, adjust=False).mean()

# Calculate VWAP (Volume Weighted Average Price)
retrieved_data['VWAP'] = (retrieved_data['Close'] * retrieved_data['Volume']).cumsum() / retrieved_data['Volume'].cumsum()

# Convert 'Datetime' column to datetime type for filtering
retrieved_data['Datetime'] = pd.to_datetime(retrieved_data['Datetime'])

retrieved_data.rename(columns={'Datetime': 'Timestamp'}, inplace=True)

retrieved_data['Day'] = retrieved_data['Timestamp'].dt.strftime('%Y-%m-%d')
day_mapping = {day: idx + 1 for idx, day in enumerate(sorted(retrieved_data['Day'].unique()))}
retrieved_data['Day'] = retrieved_data['Day'].map(day_mapping)
retrieved_data['Day'] = retrieved_data['Day'].astype(int)

# Initialize lists to store ORB, PM, and Yest values
orb_highs = []
orb_lows = []
pm_highs = []
pm_lows = []
yest_highs = []
yest_lows = []

# Loop through each distinct day in the retrieved_data dataframe
for day in retrieved_data['Day'].unique():
    # Create a daily dataframe for the current day
    daily_data = retrieved_data[retrieved_data['Day'] == day]
    
    # Calculate ORB_High and ORB_Low (first 15 minutes of open market)
    orb_data = daily_data[
        (daily_data['Timestamp'].dt.time >= datetime.strptime(market_open, '%H:%M:%S').time()) &
        (daily_data['Timestamp'].dt.time < (datetime.strptime(market_open, '%H:%M:%S') + timedelta(minutes=15)).time())
    ]
    orb_highs.append(orb_data['High'].max())
    orb_lows.append(orb_data['Low'].min())
    
    # Calculate PM_High and PM_Low (pre-market hours)
    pm_data = daily_data[
        (daily_data['Timestamp'].dt.time >= datetime.strptime(pre_market_start, '%H:%M:%S').time()) &
        (daily_data['Timestamp'].dt.time < datetime.strptime(market_open, '%H:%M:%S').time())
    ]
    pm_highs.append(pm_data['High'].max())
    pm_lows.append(pm_data['Low'].min())
    
    # Calculate Yest_High and Yest_Low (previous day's high and low)
    if len(yest_highs) == 0:  # No previous day for the first day
        yest_highs.append(None)
        yest_lows.append(None)
    else:
        prev_day_data = retrieved_data[retrieved_data['Day'] == (day - 1)]
        yest_highs.append(prev_day_data['High'].max())
        yest_lows.append(prev_day_data['Low'].min())

# Map the calculated values back to the retrieved_data dataframe
retrieved_data['ORB_High'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), orb_highs)))
retrieved_data['ORB_Low'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), orb_lows)))
retrieved_data['PM_High'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), pm_highs)))
retrieved_data['PM_Low'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), pm_lows)))
retrieved_data['Yest_High'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), yest_highs)))
retrieved_data['Yest_Low'] = retrieved_data['Day'].map(dict(zip(retrieved_data['Day'].unique(), yest_lows)))

retrieved_data['Session'] = retrieved_data['Timestamp'].dt.time.apply(
    lambda t: 'PM' if datetime.strptime(pre_market_start, '%H:%M:%S').time() <= t < datetime.strptime(market_open, '%H:%M:%S').time()
    else 'Regular Market' if datetime.strptime(market_open, '%H:%M:%S').time() <= t < datetime.strptime(market_close, '%H:%M:%S').time()
    else None
)

# Apply Signal Generation
retrieved_data = generate_signals(retrieved_data)

C:\Users\anton\AppData\Local\Temp\ipykernel_18624\3947858746.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\anton\AppData\Local\Temp\ipykernel_18624\3947858746.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\anton\AppData\Local\Temp\ipykernel_18624\3947858746.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

# Candlestick Chart with Indicators

This cell generates an interactive candlestick chart using Plotly, enriched with various technical indicators to provide insights into stock price movements. The chart includes:

- **Candlestick Data**: Visual representation of open, high, low, and close prices for each interval.
- **Indicators**:
    - **ORB High/Low**: Opening Range Breakout levels.
    - **Yesterday's High/Low**: Previous day's high and low prices.
    - **PM High/Low**: Pre-market high and low prices.
    - **8EMA**: Exponential Moving Average with a period of 8.
    - **VWAP**: Volume Weighted Average Price.
- **Session Data**:
    - Pre-market and regular market sessions are distinguished with separate line plots for their respective closing prices.

The chart is interactive, allowing users to hover over data points for detailed information and toggle between different views for better analysis.

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from generate_candlestick_df import generate_candlestick

# Generate candlestick data
candlestick_data = generate_candlestick(retrieved_data, intervalAmt) # change this to change the interval on the candlestick chart
candlestick_data.set_index('Timestamp', inplace=True)

def plot_candlestick_with_indicators(candlestick_data, simulated_data):
    # Create a figure with a single subplot
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, 
                        vertical_spacing=0.1, 
                        subplot_titles=("Candlestick Plot",))
    # Add candlestick traces
    fig.add_trace(go.Candlestick(
        x=candlestick_data.index,
        open=candlestick_data['Open'],
        high=candlestick_data['High'],
        low=candlestick_data['Low'],
        close=candlestick_data['Close'],
        name='Candlestick',
        opacity=1
    ), row=1, col=1)

    # Add indicators to the candlestick plot
    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['ORB_High'],
        mode='lines',
        name='ORB High',
        line=dict(color='green', dash='dash')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['ORB_Low'],
        mode='lines',
        name='ORB Low',
        line=dict(color='red', dash='dash')
    ), row=1, col=1)

        # Add PM and Regular Market values
    fig.add_trace(go.Scatter(
        x=simulated_data[simulated_data['Session'] == 'PM']['Timestamp'],
        y=simulated_data[simulated_data['Session'] == 'PM']['Close'],
        mode='lines',
        name='Pre-Market Value',
        line=dict(color='black', dash='dot'),
        opacity=0.7
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=simulated_data[simulated_data['Session'] == 'Regular Market']['Timestamp'],
        y=simulated_data[simulated_data['Session'] == 'Regular Market']['Close'],
        mode='lines',
        name='Regular Market Value',
        line=dict(color='steelblue'),
        opacity=0.8
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['Yest_High'],
        mode='lines',
        name="Yesterday's High",
        line=dict(color='gray', dash='dashdot')
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['Yest_Low'],
        mode='lines',
        name="Yesterday's Low",
        line=dict(color='brown', dash='dashdot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['PM_High'],
        mode='lines',
        name='PM High',
        line=dict(color='green', dash='dot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['PM_Low'],
        mode='lines',
        name='PM Low',
        line=dict(color='red', dash='dot')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['8EMA'],
        mode='lines',
        name='8EMA',
        line=dict(color='orange')
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=simulated_data['Timestamp'],
        y=simulated_data['VWAP'],
        mode='lines',
        name='VWAP',
        line=dict(color='blue')
    ), row=1, col=1)

    # Update layout for better appearance
    fig.update_layout(
        title="Simulated Stock Price with Indicators",
        xaxis_title="Timestamp",
           yaxis_title="Price",
        legend_title="Legend",
        xaxis=dict(tickangle=45),
        template="seaborn",
        hovermode='x unified',
        height=800  # Make the graph taller
    )
    # Return the figure object
    return fig

fig = plot_candlestick_with_indicators(candlestick_data, retrieved_data)

# Save the plot as an HTML file
# fig.write_html("candlestick_plot_with_indicators.html")

In [19]:
strategy_params = {
    'initial_capital': 25000,               # Starting cash balance for the strategy
    'risk_per_trade': 0.1,                  # Fraction of portfolio to risk per trade
    'open_range_min': 10,                   # Number of minutes after open to define breakout range
    'stop_loss_pct': 0.15,                  # Percentage loss that triggers a stop on a trade
    'take_profit_pct': 0.1,                 # Percentage gain that triggers profit-taking
    'expiration_days': 3,                   # Days to expiration used in Black-Scholes option pricing
    'sigma': 0.2,                           # Assumed/implied volatility used in Black-Scholes model
    'max_trades': 3,                        # Max number of trades allowed per day
    'max_minutes_in_trade': 60,             # Max time (in minutes) a position can be held
    'max_daily_loss_pct': 0.15,             # Max % of portfolio you can lose in a single day before halting trades
    'max_portfolio_cap': 4,                 # Cap portfolio growth to avoid unrealistic compounding
    'max_drawdown_pct': 0.7,                # Max intraday drawdown allowed (reset daily)
    'max_consecutive_losses': 7,            # Number of losing trades in a row before trading halts for the day
    'trailing_stop_pct': 0.25,              # Trailing stop to protect profits
    'decay_threshold': 0.03                 # Threshold for exiting if no price movement
}

In [20]:
from report_generation import generate_reports


df = pd.read_csv('df_2023_2025.csv', parse_dates=['datetime'])
df['datetime'] = pd.to_datetime(df['datetime'], utc=True).dt.tz_convert('America/New_York')
filepaths = ['candlestick_plot_with_indicators.html']
generate_reports("SPY", df, "strategy_report.xlsx", strategy_params, filepaths)


KeyboardInterrupt: 